# Location Finder
An excited new technology in the field of image datascience and radiology is the use of machine learning to locate and idenpngy specific structures in medical images in 3D space. This technology can be used to assist radiologists in diagnosing diseases and conditions, as well as to help researchers analyze large volumes of medical image data.

An example of this technology is the nnU-Net framework (linke to their nature publication: https://www.nature.com/articles/s41592-020-01008-z), which is a deep learning model for medical image segmentation. This model can be used to locate and idenpngy different organs in medical images in 3D space. In this notebook, we will demonstrate how to use a nnU-Net model trained across 120 abdomen CT and 40 MRI scans to locate and idenpngy structures in your image stack from either image technology.

The output of the model will be a prediction on the location of the structures in the image stack. The prediction will be in the form of a description that describes the location of the structures in the image stack. The prediction will be displayed in the notebook.

This notebook will guide you through the process of downloading the required files, uploading your image stack, formating your uploaded image stack and running the model to generate a prediction. To get started, run the cells below and follow the instructions.

# Download required files
Before we can use the model, we need to download the required files. The files include the trained model weights and configuration. The files are stored in a ZIP archive, which we will download and automatically extract. When the extraction is complete, the script will automatically set the environment variables required by the prediction logarithm nnU-Net framework.

In [1]:
import os
import zipfile

!pip install ipywidgets nnUNetv2 requests

import requests
# Define the URL and output file name
url = "https://wustl.box.com/shared/static/12gz6cy9n5isi5e3b6zcxgow6g9uf9v8.zip"
output_file = "Results_Folder.zip"
extracted_folder = "Results_Folder"

# Define the URL and file paths
url = "https://wustl.box.com/shared/static/12gz6cy9n5isi5e3b6zcxgow6g9uf9v8.zip"
output_file = "Results_Folder.zip"
extracted_folder = "Results_Folder"

# Download the file using requests
def download_file(url, output_file):
    print("Starting download...")
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise an exception for HTTP errors
        with open(output_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    print(f"File downloaded successfully: {output_file}")

# Unzip the downloaded file
def unzip_file(zip_file, extract_to):
    print(f"Unzipping file: {zip_file}")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Files extracted to: {extract_to}")

# Perform the download and extraction
if os.path.exists(extracted_folder):
    # Delete existing files
    for root, dirs, files in os.walk(extracted_folder, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    os.rmdir(extracted_folder)
    print(f"Deleted existing folder: {extracted_folder}")

if os.path.exists(output_file):
    os.remove(output_file)
    print(f"Deleted existing file: {output_file}")

download_file(url, output_file)
unzip_file(output_file, extracted_folder)


# Get the absolute path of the extracted folder
extracted_path = os.path.abspath(extracted_folder)

extracted_path = os.path.join(extracted_path, "Results_Folder")
# Set environment variables
os.environ['nnUNet_raw'] = extracted_path
os.environ['nnUNet_results'] = extracted_path
os.environ['nnUNet_preprocessed'] = extracted_path

# Print the environment variables to verify
print(f"nnUNet_raw={os.environ['nnUNet_raw']}")
print(f"nnUNet_results={os.environ['nnUNet_results']}")
print(f"nnUNet_preprocessed={os.environ['nnUNet_preprocessed']}")

Defaulting to user installation because normal site-packages is not writeable
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Deleted existing folder: Results_Folder
Deleted existing file: Results_Folder.zip
Starting download...
File downloaded successfully: Results_Folder.zip
Unzipping file: Results_Folder.zip
Files extracted to: Results_Folder
nnUNet_raw=/home/warehouse/c.b.hanan/Results_Folder/Results_Folder
nnUNet_results=/home/warehouse/c.b.hanan/Results_Folder/Results_Folder
nnUNet_preprocessed=/home/warehouse/c.b.hanan/Results_Folder/Results_Folder


# Upload the image stack
Upload and select the folder containing the image stack you want to analyze. The image stack should be in a folder containing the image files. You can upload files to EIT by going to the filebrowser tab at [linuxlab.engr.wust.edu](https://linuxlab.engr.wustl.edu) and clicking the blue upload button. Once you have uploaded the image stack, run the cell below and select the folder containing the image stack.

In [7]:
import os
import ipywidgets as widgets
from IPython.display import display

# Initialize the current path
current_path = os.getcwd()  # This will hold the current directory path

# Create widgets
path_display = widgets.Text(
    value=current_path,
    description='Path:',
    layout=widgets.Layout(width='800px')
)

output = widgets.Output()

def display_directories(path):
    """Display only directories in the given path as clickable rectangles."""
    items = [item for item in os.listdir(path) if os.path.isdir(os.path.join(path, item))]
    buttons = []

    for item in items:
        item_path = os.path.join(path, item)
        button = widgets.Button(
            description=item,
            layout=widgets.Layout(width='auto', height='30px')
        )
        button.style.button_color = '#007bff'
        button.on_click(lambda b, p=item_path: handle_directory_click(p))
        buttons.append(button)

    return widgets.VBox(buttons)

def count_tif_files(path):
    """Count the number of .tif files in the given directory."""
    return len([f for f in os.listdir(path) if f.endswith('.tif')])

def handle_directory_click(directory_path):
    """Handle clicks on directories."""
    global current_path  # Make current_path accessible globally
    current_path = directory_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

    # Check for .tif files and display a warning if there are less than two
    tif_count = count_tif_files(directory_path)
    output.clear_output()
    with output:
        if tif_count <= 1:
            print(f"Warning: The directory '{os.path.basename(directory_path)}' contains {tif_count} .tif file(s).")
        else:
            print(f"The directory '{os.path.basename(directory_path)}' contains enough ({tif_count}) .tif file(s).")

def navigate_to_parent_directory(_):
    """Navigate to the parent directory."""
    global current_path  # Make current_path accessible globally
    parent_path = os.path.dirname(current_path)
    current_path = parent_path  # Update global variable
    path_display.value = current_path
    refresh_directory_view(current_path)

def refresh_directory_view(path):
    """Refresh the directory view."""
    directory_view.children = [display_directories(path)]

# Button to go to the parent directory
parent_button = widgets.Button(
    description='Go to Parent Directory',
    layout=widgets.Layout(width='auto', height='30px')
)
parent_button.style.button_color = '#007bff'
parent_button.on_click(navigate_to_parent_directory)

# Initialize the directory view
directory_view = widgets.VBox(children=[display_directories(current_path)])

# Layout the widgets
navigation_box = widgets.VBox([path_display, parent_button, directory_view, output])
display(navigation_box)

global directory_path


# If the current path stops updating, run this cell again

# Format the image stack
Using the folder path selected above, we will format the image stack to be compatible with the nnU-Net framework. The formatting will include converting the image stack to .nii.gz format and saving it to a temporary folder. The image stack has to be in .nii.gz format for the nnU-Net framework to process it as that is the same format the model was trained on. Fortunately, it is easy to convert the image stack to .nii.gz format  using the cell below

In [8]:
import os
import nibabel as nib
import numpy as np
import tifffile

def count_tiff_files(directory_path):
    """Count the number of .tif files in the given directory."""
    return len([f for f in os.listdir(directory_path) if f.endswith('.tif')])

def gather_and_sort_images(folder_path):
    """Gathers and sorts TIFF image files from a directory."""
    image_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')])
    return image_files

def read_and_stack_images(files):
    """Reads TIFF image files and stacks them into a numpy array."""
    try:
        image_stack = [tifffile.imread(file) for file in files]
        return np.stack(image_stack, axis=0)  # Adjust to stack along the first axis

    except Exception as e:
        print(f"Error reading TIFF files: {e}")
        return None

def save_nifti(image_stack, output_path):
    """Saves the numpy stack as a NIfTI file with an identity affine matrix."""
    try:
        nifti_image = nib.Nifti1Image(image_stack, affine=np.eye(4))
        nib.save(nifti_image, output_path)
        print(f"Converted TIFF stack saved as .nii.gz file in: {output_path}")
    except Exception as e:
        print(f"Error saving NIfTI file: {e}")

def convert_tiff_to_nifti(tiff_folder, output_folder):
    """Converts a set of TIFF images in a folder into a single NIfTI file."""
    if not os.path.isdir(tiff_folder):
        print("The specified TIFF folder does not exist.")
        return
    tiff_files = gather_and_sort_images(tiff_folder)
    if not tiff_files:
        print("No TIFF files found in the specified directory.")
        return
    image_stack = read_and_stack_images(tiff_files)
    if image_stack is None:
        return
    # Generating the output filename based on directory structure
    specific_temp_folder = os.path.join(tiff_folder, "tempFolder")
    os.makedirs(specific_temp_folder, exist_ok=True)

    parts = tiff_folder.strip('/').split('/')
    output_file_name = f"amos_0001_0000.nii.gz"
    output_file_path = os.path.join(specific_temp_folder, output_file_name)
    save_nifti(image_stack, output_file_path)

def main(current_path):
    if not os.path.isdir(current_path) or not count_tiff_files(current_path):
        print("Warning: 'current_path' is invalid or contains no TIFF files. Please verify the path.")

    else:
        # Specify the path for temporary files
        convert_tiff_to_nifti(current_path, os.path.join(current_path, "tempFolder"))
        print(f"Temporary files are stored in: {os.path.join(current_path, 'tempFolder')}")

if __name__ == '__main__':
    main(current_path)

Converted TIFF stack saved as .nii.gz file in: /home/warehouse/c.b.hanan/region_2_ct_3/tempFolder/amos_0001_0000.nii.gz
Temporary files are stored in: /home/warehouse/c.b.hanan/region_2_ct_3/tempFolder


# Running the Model on your Image Stack
The `nnUNetv2_predict` command calls the nnU-Net framework to predict the location of the structures in the image stack. Below are the details of the flags used in the command:

- `-i`: Specifies the input folder containing the image stack.
- `-o`: Specifies the output folder to save the results.
- `-d`: Specifies the dataset ID (provided).
- `-c`: Specifies the resolution of the model.
- `--save_probabilities`: Saves the predicted probabilities for each class.

In [9]:
import os
import shutil
import sys

def find_nnunet_predict():
    home_dir = os.path.expanduser("~")
    local_bin_dir = os.path.join(home_dir, ".local", "bin")
    nnunet_path = shutil.which("nnUNetv2_predict", path=local_bin_dir)
    if nnunet_path is None:
        raise FileNotFoundError("nnUNetv2_predict not found in ~/.local/bin. Ensure it is installed and accessible.")
    return nnunet_path

def main(current_path, output_folder):
    nnunet_path = find_nnunet_predict()
    specific_temp_folder = os.path.join(current_path, "tempFolder")

    # Ensure the input folder exists and is not empty
    if not os.path.exists(specific_temp_folder) or not os.listdir(specific_temp_folder):
        print(f"No input files found in {specific_temp_folder}. Please check the directory.")
        return

    # Run the nnUNetv2 prediction command
    command = f"{nnunet_path} -i {specific_temp_folder} -o {output_folder} -d 123 -c 3d_fullres --save_probabilities"
    os.system(command)

if __name__ == '__main__':
    output_folder = os.path.join(current_path, "outputFolder1")
    main(current_path, output_folder)


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting amos_0001:
perform_everything_on_device: True


100%|██████████| 16/16 [00:05<00:00,  2.99it/s]


sending off prediction to background worker for resampling and export
done with amos_0001


In [11]:
import os
import nibabel as nib
import numpy as np
import json

# Define the output folder where your .nii.gz files are stored
output_folder = os.path.join(current_path, "outputFolder1")

# Find the .nii.gz files in the specified output folder
nii_files = [f for f in os.listdir(output_folder) if f.endswith('.nii.gz')]
if not nii_files:
    raise FileNotFoundError("No .nii.gz files found in the output folder.")
file_path = os.path.join(output_folder, nii_files[0])

# Load the .nii.gz file
img = nib.load(file_path)

# Convert the image data to a NumPy array
image_data = img.get_fdata()

# Get unique region numbers, excluding background (0) and numbers greater than 12
unique_regions = np.unique(image_data)
filtered_regions = unique_regions[(unique_regions > 0) & (unique_regions <= 12)]
print("Filtered regions:", filtered_regions)

# Ensure the nnUNet_raw environment variable is set and construct the path to the plan2.json
nnUNet_raw = os.environ.get('nnUNet_raw')
if not nnUNet_raw:
    raise EnvironmentError("Environment variable 'nnUNet_raw' is not set.")
nnUNet_raw = os.path.join(nnUNet_raw, "Dataset123_Foo", "nnUNetTrainer__nnUNetPlans__3d_fullres")
plan2_path = os.path.join(nnUNet_raw, "plan2.json")
if not os.path.exists(plan2_path):
    raise FileNotFoundError(f"plan2.json not found in {nnUNet_raw}.")

# Load the plan2.json file
with open(plan2_path, 'r') as f:
    plan2_data = json.load(f)

# Print the strings corresponding to the region numbers
for region in filtered_regions:
    region_str = plan2_data.get(str(int(region)), "Unknown region")
    print(f"Region {int(region)}: {region_str}")

Filtered regions: [6.]
Region 6: This large organ likely spans the upper right abdomen, possibly protected by the rib cage.
